In [ ]:
from deploy.preprocess import decode_image
from deploy.infer import Detector, DetectorPicoDet, PredictConfig, print_arguments, get_test_images, bench_log
from deploy.keypoint_infer import KeyPointDetector, PredictConfig_KeyPoint
from deploy.visualize import visualize_pose
from deploy.benchmark_utils import PaddleInferBenchmark
from deploy.utils import get_current_memory_mb
from deploy.keypoint_postprocess import translate_to_ori_images
from pathlib import Path
from typing import Literal, List, Dict, TypedDict, Any, cast
from pydantic import BaseModel
from numpy.typing import NDArray
import numpy as np
import cv2
import cv2 as cv
from cv2.typing import MatLike
from loguru import logger
import matplotlib.pyplot as plt
import matplotlib

class DetectionResult(TypedDict):
    # images
    # box count
    boxes_num: NDArray[np.integer]
    # images
    # bounding box
    #  - x y w h (?) or x1 y1 x2 y2
    boxes:NDArray[np.floating]

class KeyPointDetectionResult(TypedDict):
    # images
    # key points
    # x y possibility (?)
    keypoint: NDArray[np.floating]
    # images
    # bounding boxes
    #  - x y w h (?) or x1 y1 x2 y2
    bboxes: NDArray[np.integer]

class ImageInfo(TypedDict):
    # im.shape[:2]
    shape: NDArray[np.float32]
    # scale_factor
    scale_factor: NDArray[np.float32]

DeviceType = Literal["GPU", "CPU", "XPU"]
# paddle/trt_fp32/trt_fp16/trt_int8
RunMode = Literal["paddle", "trt_fp32", "trt_fp16", "trt_int8"]

MODELS_DIR = Path("models")
DETECTION_MODEL_DIR = MODELS_DIR / "picodet_v2_s_320_pedestrian"
KEYPOINT_MODEL_DIR = MODELS_DIR / "tinypose_256x192"
DEVICE: DeviceType = "GPU"
RUN_MODE:RunMode = "paddle"
USE_DARK_POSE = True


In [ ]:
class DetectionParams(BaseModel):
    detection_thres:float
    key_points_visual_thresh:float

In [ ]:
def predict_with_given_det(
        image: MatLike,
        det_res: DetectionResult,
        keypoint_detector: KeyPointDetector,
        run_benchmark: bool = False) -> KeyPointDetectionResult:
    keypoint_res = {}

    rec_images, records, det_rects = keypoint_detector.get_person_from_rect(
        image, det_res)

    if len(det_rects) == 0:
        keypoint_res['keypoint'] = [[], []]
        return keypoint_res

    keypoint_vector = []
    score_vector = []

    rect_vector = det_rects
    keypoint_results = keypoint_detector.predict_image(rec_images,
                                                       run_benchmark,
                                                       repeats=10,
                                                       visual=False)
    keypoint_vector, score_vector = translate_to_ori_images(
        keypoint_results, np.array(records))
    keypoint_res['keypoint'] = [
        keypoint_vector.tolist(),
        score_vector.tolist()
    ] if len(keypoint_vector) > 0 else [[], []]
    keypoint_res['bbox'] = rect_vector
    return keypoint_res

In [ ]:
# if the arch is not PicoDet
detector = Detector(DETECTION_MODEL_DIR, device=DEVICE, run_mode=RUN_MODE)
kp_detector = KeyPointDetector(KEYPOINT_MODEL_DIR,
                               device=DEVICE,
                               run_mode=RUN_MODE,
                               use_dark=USE_DARK_POSE)


# detector needs RGB with dtype uint8 (8UC3)
def preprocess(frame: MatLike) -> tuple[MatLike, ImageInfo]:
    assert frame.dtype == np.uint8, "Image pixel type should be uint8_t"
    assert frame.shape[2] == 3, "The image does not have 3 channels"
    im = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    info: ImageInfo = {
        "shape": np.array(im.shape[:2], dtype=np.float32),
        "scale_factor": np.array([1., 1.], dtype=np.float32)
    }
    return im, info


def predict(image: MatLike, detector: Detector,
            topdown_kp_detector: KeyPointDetector, params: DetectionParams):
    detections = detector.predict_image([image], visual=False)
    det_res = detector.filter_box(detections, params.detection_thres)
    det_res = cast(DetectionResult, det_res)
    kp_res = predict_with_given_det(image, det_res, topdown_kp_detector, 1)
    kp_res = cast(KeyPointDetectionResult, kp_res)
    return det_res, kp_res

In [ ]:
img = cv.imread("test.jpeg")

In [ ]:
rgb, info = preprocess(img)
plt.imshow(rgb)
plt.show()

In [ ]:
params = DetectionParams(detection_thres=0.5, key_points_visual_thresh=0.5)
det, kp = predict(rgb, detector, kp_detector, params)
display(det)
display(kp)

In [ ]:
def visualize(
    frame: MatLike,
    results: KeyPointDetectionResult,
    visual_thresh: float = 0.6,
    bb_thickness:int = 1,
    kp_stick_width:int = 2,
):
    import math
    skeletons, scores = results['keypoint']
    skeletons = np.array(skeletons)
    kpt_nums = 17
    if len(skeletons) > 0:
        kpt_nums = skeletons.shape[1]
    if kpt_nums == 17:  #plot coco keypoint
        EDGES = [(0, 1), (0, 2), (1, 3), (2, 4), (3, 5), (4, 6), (5, 7),
                 (6, 8), (7, 9), (8, 10), (5, 11), (6, 12), (11, 13), (12, 14),
                 (13, 15), (14, 16), (11, 12)]
    else:  #plot mpii keypoint
        EDGES = [(0, 1), (1, 2), (3, 4), (4, 5), (2, 6),
                 (3, 6), (6, 7), (7, 8), (8, 9), (10, 11), (11, 12), (13, 14),
                 (14, 15), (8, 12), (8, 13)]
    NUM_EDGES = len(EDGES)

    colors = [[255, 0, 0], [255, 85, 0], [255, 170, 0], [255, 255, 0], [170, 255, 0], [85, 255, 0], [0, 255, 0], \
            [0, 255, 85], [0, 255, 170], [0, 255, 255], [0, 170, 255], [0, 85, 255], [0, 0, 255], [85, 0, 255], \
            [170, 0, 255], [255, 0, 255], [255, 0, 170], [255, 0, 85]]
    cmap = matplotlib.colormaps.get_cmap('hsv')
    plt.figure()

    img = cv2.imread(frame) if type(frame) == str else frame

    color_set = results['colors'] if 'colors' in results else None

    if 'bbox' in results:
        bboxs = results['bbox']
        for j, rect in enumerate(bboxs):
            xmin, ymin, xmax, ymax = rect
            color = colors[0] if color_set is None else colors[color_set[j] %
                                                               len(colors)]
            cv2.rectangle(img, (xmin, ymin), (xmax, ymax), color, bb_thickness)

    canvas = img.copy()
    for i in range(kpt_nums):
        for j in range(len(skeletons)):
            if skeletons[j][i, 2] < visual_thresh:
                continue
            if True:
                color = colors[i] if color_set is None else colors[
                    color_set[j] % len(colors)]
            else:
                color = get_color(ids[j])

            cv2.circle(canvas,
                       tuple(skeletons[j][i, 0:2].astype('int32')),
                       2,
                       color,
                       thickness=-1)

    for i in range(NUM_EDGES):
        for j in range(len(skeletons)):
            edge = EDGES[i]
            if skeletons[j][edge[0], 2] < visual_thresh or skeletons[j][
                    edge[1], 2] < visual_thresh:
                continue

            cur_canvas = canvas.copy()
            X = [skeletons[j][edge[0], 1], skeletons[j][edge[1], 1]]
            Y = [skeletons[j][edge[0], 0], skeletons[j][edge[1], 0]]
            mX = np.mean(X)
            mY = np.mean(Y)
            length = ((X[0] - X[1])**2 + (Y[0] - Y[1])**2)**0.5
            angle = math.degrees(math.atan2(X[0] - X[1], Y[0] - Y[1]))
            polygon = cv2.ellipse2Poly(
                (int(mY), int(mX)), (int(length / 2), kp_stick_width), int(angle),
                0, 360, 1)
            if True:
                color = colors[i] if color_set is None else colors[
                    color_set[j] % len(colors)]
            else:
                color = get_color(ids[j])
            cv2.fillConvexPoly(cur_canvas, polygon, color)
            canvas = cv2.addWeighted(canvas, 0.4, cur_canvas, 0.6, 0)
    return canvas


In [ ]:
res_img = visualize(rgb, kp, params.key_points_visual_thresh, bb_thickness=5, kp_stick_width=10)

In [ ]:
type(res_img)

In [ ]:
plt.imshow(res_img)
plt.show()